<a href="https://colab.research.google.com/github/RBorczuch/PlatesOCR/blob/main/license_plates_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import shutil

# Define the path to the folder to be deleted
folder_path = '/content/dataset'

# Check if the folder exists and delete it
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")
else:
    print(f"Folder '{folder_path}' does not exist.")


Folder '/content/dataset' has been deleted.


In [1]:
import os
import cv2
import json
import requests
import zipfile
import numpy as np

# Function to download and extract dataset
def download_and_extract(url, extract_to='.'):
    local_filename = url.split('/')[-1].split('?')[0] + '.zip'
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    with zipfile.ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Function to display directory structure
def display_directory_structure(root_dir):
    for root, dirs, files in os.walk(root_dir):
        level = root.replace(root_dir, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

# Download dataset in COCO format from RoboFlow
dataset_url = "https://universe.roboflow.com/ds/kNwo920tMC?key=7xwvlsYPjp"  # Replace with your RoboFlow download link
download_and_extract(dataset_url, extract_to='/content/dataset')

# Display directory structure
display_directory_structure('/content/dataset')

# Define paths to the directories
base_dir = '/content/dataset'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
output_dir = os.path.join(base_dir, 'extracted_objects')

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Target dimensions
TARGET_WIDTH = 400
TARGET_HEIGHT = 100

# Function to apply perspective transformation
def apply_perspective_transform(image, src_points, dst_points, target_width, target_height):
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    return cv2.warpPerspective(image, M, (target_width, target_height))

# Function to process annotations and extract objects
def process_annotations(image_dir, annotations_file, output_dir):
    if not os.path.exists(annotations_file):
        print(f"Annotation file not found at {annotations_file}")
        return

    with open(annotations_file, 'r') as f:
        coco_data = json.load(f)

    # Map image IDs to file names
    id_to_filename = {image['id']: image['file_name'] for image in coco_data['images']}

    # Process each annotation
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        segmentation = annotation['segmentation']
        class_id = annotation['category_id']

        image_path = os.path.join(image_dir, id_to_filename[image_id])
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found or unable to load: {image_path}")
            continue

        original_height, original_width, _ = image.shape
        print(f"Processing image: {image_path}, original size: ({original_width}, {original_height})")

        # Process each segmentation polygon
        for segment in segmentation:
            segment = np.array(segment).reshape((-1, 2))
            segment = segment.astype(np.float32)

            # Create mask from segmentation
            mask = np.zeros((original_height, original_width), dtype=np.uint8)
            cv2.fillPoly(mask, [segment.astype(np.int32)], 255)

            # Extract object using mask
            obj_image = cv2.bitwise_and(image, image, mask=mask)

            # Bounding box around the object
            x, y, w, h = cv2.boundingRect(segment)
            extracted_obj = obj_image[y:y+h, x:x+w]

            if extracted_obj.size == 0:
                print(f"Extracted object is empty for annotation id {annotation['id']}")
                continue

            # Define source and destination points for perspective transform
            src_points = np.float32([
                [x, y],
                [x + w, y],
                [x + w, y + h],
                [x, y + h]
            ])
            dst_points = np.float32([
                [0, 0],
                [TARGET_WIDTH - 1, 0],
                [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
                [0, TARGET_HEIGHT - 1]
            ])

            # Apply perspective transform
            final_object = apply_perspective_transform(obj_image, src_points, dst_points, TARGET_WIDTH, TARGET_HEIGHT)

            # Save the extracted object
            output_path = os.path.join(output_dir, f'object_{image_id}_{class_id}_{annotation["id"]}.jpg')
            cv2.imwrite(output_path, final_object)
            print(f"Saved extracted object to {output_path}")

# Process train and valid directories
train_annotations_file = '/content/dataset/train/_annotations.coco.json'
valid_annotations_file = '/content/dataset/valid/_annotations.coco.json'

process_annotations(train_dir, train_annotations_file, output_dir)
process_annotations(valid_dir, valid_annotations_file, output_dir)

print("Objects extracted and saved successfully.")


dataset/
    README.dataset.txt
    README.roboflow.txt
    train/
        01_436_jpg.rf.3e87c9115c229728bdea808e99b29684.jpg
        01_734_jpg.rf.9f7a574effa4f6116ae5e2ee8fb7ab4f.jpg
        01_537_jpg.rf.e62f14e18632d3f6927a6d21398052a2.jpg
        01_734_jpg.rf.0fcaedb3bc42b430ac78eff86e7ccf50.jpg
        01_894_jpg.rf.30cd7846886f06fe46a3929fed46aab0.jpg
        01_88_jpg.rf.0a50bc4bd91009d5cb2a2c844a5eee37.jpg
        01_489_jpg.rf.1e24f0b82e94d3e6f8cd486e93a62bfe.jpg
        01_0_jpg.rf.c109dd6005fd1ae5b13e7c53ce0508b0.jpg
        01_63_jpg.rf.3402f364c1f719542e846f87d4a9a392.jpg
        01_595_jpg.rf.f794edc49fd6b717c914c2538019e76d.jpg
        01_887_jpg.rf.11962beb9df42518d6731744e6f120e8.jpg
        01_158_jpg.rf.33ad51518613970afea4fb85d9b1e564.jpg
        01_116_jpg.rf.3e403a74e8fc4cbf2a9fd36088411736.jpg
        01_588_jpg.rf.0d5173e79c8b7609568f44e26286eeda.jpg
        01_159_jpg.rf.5f3f33ad575e4a94e0f61a357951db06.jpg
        01_423_jpg.rf.0b128369fac0eb79bacfa13523caf0

In [2]:
import os
import cv2
import numpy as np

# Define paths to the directories
input_dir = '/content/dataset/extracted_objects'
output_dir = '/content/dataset/extracted_objects_p'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)


# Function to apply histogram equalization
def equalize_histogram_color(image):
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Split the LAB image to different channels
    l, a, b = cv2.split(lab)

    # Apply histogram equalization on the L-channel
    l_eq = cv2.equalizeHist(l)

    # Merge the channels back
    lab_eq = cv2.merge((l_eq, a, b))

    # Convert back to BGR color space
    image_eq = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

    return image_eq

# Function to process images
def process_images(input_dir, output_dir):
    for image_name in os.listdir(input_dir):
        image_path = os.path.join(input_dir, image_name)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found or unable to load: {image_path}")
            continue

        # Apply histogram equalization
        image_eq = equalize_histogram_color(image)

        # Convert to grayscale
        gray_image = cv2.cvtColor(image_eq, cv2.COLOR_BGR2GRAY)

        # Threshold the image to get only black areas
        _, thresh_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY_INV)

        # Create a mask for the black areas
        mask = cv2.inRange(image_eq, (0, 0, 0), (130, 200, 200))

        # Set all non-black areas to white
        image_eq[mask == 0] = [255, 255, 255]
        #image_eq[mask == 1] = [0, 0, 0]
        image_eq = cv2.cvtColor(image_eq, cv2.COLOR_BGR2GRAY)
        #kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        #image_eq= cv2.dilate(image_eq, kernel, iterations=1)
        #image_eq = cv2.erode(image_eq, kernel, iterations=1)

        # Save the processed image
        output_path = os.path.join(output_dir, image_name)
        cv2.imwrite(output_path, image_eq)
        print(f"Saved processed image to {output_path}")

# Process images
process_images(input_dir, output_dir)

print("Images processed and saved successfully.")


Saved processed image to /content/dataset/extracted_objects_p/object_161_1_158.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_180_1_177.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_4_1_4.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_149_1_146.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_214_1_211.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_19_1_19.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_101_1_99.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_177_1_174.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_136_1_133.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_54_1_54.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_137_1_134.jpg
Saved processed image to /content/dataset/extracted_objects_p/object_269_1_266.jpg
Saved process

In [9]:
!pip install easyocr

In [10]:
import os
import cv2
import easyocr

# Ensure the output directory exists
output_dir = '/content/dataset/extracted_objects_ocr'
os.makedirs(output_dir, exist_ok=True)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)  # Set gpu=True if you have a GPU available

# Function to process images and save them with OCR-detected names
def process_images_with_ocr(input_dir, output_dir):
    for image_name in os.listdir(input_dir):
        image_path = os.path.join(input_dir, image_name)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found or unable to load: {image_path}")
            continue

        # Perform OCR on the image
        result = reader.readtext(image)

        # Construct the new name based on OCR results
        ocr_texts = [res[1] for res in result]
        ocr_name = "_".join(ocr_texts).replace(" ", "_")  # Join texts with underscores and replace spaces

        # Save the image with the new name
        output_path = os.path.join(output_dir, f"{ocr_name}.jpg")
        cv2.imwrite(output_path, image)
        print(f"Saved image to {output_path}")

# Define the input directory
input_dir = '/content/dataset/extracted_objects_p'

# Process images
process_images_with_ocr(input_dir, output_dir)

print("Images processed and saved with OCR-detected names successfully.")


Saved image to /content/dataset/extracted_objects_ocr/01_EC_7B.jpg
Saved image to /content/dataset/extracted_objects_ocr/J1bGz_01.jpg
Saved image to /content/dataset/extracted_objects_ocr/DA_866.jpg
Saved image to /content/dataset/extracted_objects_ocr/Aot_DFz_96.jpg
Saved image to /content/dataset/extracted_objects_ocr/Jo1_CH_7X.jpg
Saved image to /content/dataset/extracted_objects_ocr/01_Al?_161.jpg
Saved image to /content/dataset/extracted_objects_ocr/Q1_Lb1078.jpg
Saved image to /content/dataset/extracted_objects_ocr/8805.jpg
Saved image to /content/dataset/extracted_objects_ocr/Lbl_44Z.jpg
Saved image to /content/dataset/extracted_objects_ocr/L0LaBe_95.jpg
Saved image to /content/dataset/extracted_objects_ocr/_41_91.jpg
Saved image to /content/dataset/extracted_objects_ocr/Hon_M_425.jpg
Saved image to /content/dataset/extracted_objects_ocr/.jpg
Saved image to /content/dataset/extracted_objects_ocr/01_@RT_64.jpg
Saved image to /content/dataset/extracted_objects_ocr/.jpg
Saved image

In [11]:
import shutil
import os
from google.colab import files

# Definiowanie ścieżki do folderu, który chcesz spakować
folder_to_zip = '/content/dataset/extracted_objects_ocr'

# Definiowanie nazwy pliku zip
output_filename = '/content/extracted_objects_ocr.zip'

# Tworzenie pliku ZIP
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)

# Pobranie pliku ZIP
files.download(output_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>